In [23]:
import chess.pgn

MIN_ELO = 1800
GENERATE_GAMES = 50

In [24]:
# read games from a .pgn file

def process_game(game: chess.pgn.Game) -> None:
    # iterate over game moves and store fen in a list
    fen_list = []
    board = game.board()
    for move in game.mainline_moves():
        board.push(move)
        fen_list.append(" ".join(board.fen().split(" ")[:2]))
    
    # return list of fens
    return fen_list


moves = []
with open("../data/games.pgn") as pgn:
    while len(moves) < GENERATE_GAMES:
        game = chess.pgn.read_game(pgn)

        # if elo is too low, skip game
        elo = (int(game.headers["WhiteElo"]) + int(game.headers["BlackElo"])) / 2
        if elo > MIN_ELO:
          continue

        # process game
        fens = process_game(game)
        moves.extend(fens)

# elminiates duplicates
moves = list(set(moves))


# Calculations

run stockfish on every game and get the move evaluation

In [27]:
import chess
from tqdm.notebook import tqdm

# create a stockfish engine
import chess.engine
stockfish = chess.engine.SimpleEngine.popen_uci("../stockfish.avx2")

# evaluate position using stockfish
def evaluate_position(fen: str) -> float:
    board = chess.Board(fen)
    info = stockfish.analyse(board, chess.engine.Limit(time=0.2))
    return info["score"].white().score(mate_score=1000)

# get evaluation for each position
evaluations = [evaluate_position(fen) for fen in tqdm(moves)]

# close stockfish engine
stockfish.quit()

  0%|          | 0/55 [00:00<?, ?it/s]

# dataset building

Create numpy arrays from chess positions.  

One-hot encode chess positions and white-black onehot.  

Array of float predictions from stockfish

In [61]:
import numpy as np

def fen_to_npy(fen: str) -> np.array:
    board = chess.Board(fen)
    npy = np.zeros((12, 8,8))
    for square, piece in board.piece_map().items():
        x = square%8
        y = square//8
        piece_owner = 0 if piece.color == chess.WHITE else 1
        piece_type = piece.piece_type -1 + 6*piece_owner

        npy[piece_type][y][x] = 1
    return npy

def fen_to_turn(fen: str) -> int:
    return [1, 0] if fen.split(" ")[1] == "w" else [0, 1]

In [63]:
one_hot_boards = np.array([fen_to_npy(fen) for fen in moves])
turns = np.array([fen_to_turn(fen) for fen in moves])
evaluations = np.array(evaluations)

# save data to disk

use numpy.save()

In [64]:
# save data
np.save("../data/one_hot_boards.npy", one_hot_boards)
np.save("../data/turns.npy", turns)
np.save("../data/evaluations.npy", evaluations)